# Reviewer Randomizer

**Inputs:**
1. Number of people to be assigned to each applicant.
2. List of reviewer names.
3. List of applicant names.

**Outputs:**
1. Table of applicants with randomly assigned reviewers.
2. Table of reviewers with assigned applicants to review.
3. Text file with reviewer names and their assigned applicants.

*(Both output tables contain the same information in different format.)*

In [ ]:
# Load packages
from random import randrange
from math import floor
import re
from google.colab import files

### Enter inputs:
* **Number of eyes on each application** (`eyes`)
* **Reviewer names** (`reviewers`) - each reviewer on a new line; 3 quotes (`"""`) at the beginning and end of the list. Names can be first and last or just first; first and last names separated by space or a tab (doesn't matter). Role should be listed after each name ("faculty", "postdoc", "student", etc.). You should be able to paste the list in from excel if desired.
* **Applicant names** (`applicants`) - same layout as reviewers (but no role required)

In [ ]:
# Number of people assigned to each applicant.
eyes = 3

In [ ]:
# Input list of reviewers & their roles. 
# Need 3 quotation marks at beginning and end of LIST only. 
# Each reviewer should be on a new line.
# Case does not matter.
reviewers = """Gary Westbrook   Faculty
Tianyi  Mao  Faculty
Kevin Wright    Faculty
Larry Trussell  Faculty
Henrique vonGersdorf    Faculty
Eric Schnell    Faculty
Kelly Monk  Faculty
Yessica Santana Student
Nina Luong  Student
Jennifer Jahncke    Student
Marissa Co  Postdoc"""
reviewers = reviewers.replace("\t", " ").split("\n")

In [ ]:
# Input list of applicants. Need 3 quotation marks at beginning and end of list only. Each reviewer should be on a new line.
applicants = """A	B
C	D
E	F
G	H
I	J
K	L
M	N
O	P
Q	R
S	T
U	V
W	X
Y	Z"""
applicants = applicants.replace("\t", " ").split("\n")

### Run Script, Export Excel Sheets

In [ ]:
## ---------------------------- ##
## Calculate assignment lengths ##
## ---------------------------- ##
reviewer_num = len(reviewers)
applicant_num = len(applicants)
min_num = floor(applicant_num * eyes / reviewer_num)
remainder = (applicant_num * eyes) % reviewer_num
# how many reviewers will be assigned the max_num, how many min_num?
assign_max = remainder
assign_min = reviewer_num - remainder


def assign_reviewer():
    ## ---------------------------- ##
    ## Build up empty lists/dicts   ##
    ## ---------------------------- ##
    reviewer_counts = {} # keep track of how many students each reviewer is assigned
    reviewer_dict = {} # keep a list of each applicant assigned to each reviewer
    reviewer_list = [] # list of all reviewers
    faculty_list = [] # subset list of faculty reviewers
    trainee_list = [] # subset list of trainee (non-faculty) reviewers
    max_list = [] # reviewers to be assigned the max number of applicants
    min_list = [] # reviewers to be assigned the min number of applicants
    # parse reviewer names, roles; build up reviewer lists/dictionaries
    for reviewer in reviewers:
        # parse name, role
        for found in re.finditer(r"^([\w\s]+)\s(\w+)$",reviewer):
            reviewer_counts[found.group(1).rstrip()] = 0
            reviewer_dict[found.group(1).rstrip()] = []
            reviewer_list.append(found.group(1).rstrip())
            # assign to faculty_list or trainee_list
            if found.group(2).lower() == "faculty":
                faculty_list.append(found.group(1).rstrip())
            else:
                trainee_list.append(found.group(1).rstrip())
            # assign to either max_list or min_list
            list_assign = randrange(2)
            if list_assign == 0 and len(min_list) >= assign_min:
                list_assign = 1
            elif list_assign == 1 and len(max_list) >= assign_max:
                list_assign = 0
            if list_assign == 0:
                min_list.append(found.group(1).rstrip())
            elif list_assign == 1:
                max_list.append(found.group(1).rstrip())

    applicant_dict = {} # keep a list of each reviewer assigned to each applicant
    applicant_counts = {} # keep track of how many reviewers have been assigned
    for applicant in applicants:
        applicant_counts[applicant] = 0
        applicant_dict[applicant] = []

    ## ------------------------------ ##
    ## Assign applicants to reviewers ##
    ## ------------------------------ ##
    
    # trainees first
    i = 0
    for trainee in trainee_list:
        rev_list = []
        if trainee in min_list:
            assign_num = min_num
        elif trainee in max_list:
            assign_num = min_num + 1
        for x in range(assign_num):
            i += 1
            temp = applicants[randrange(applicant_num)]
            while temp in rev_list or applicant_counts[temp] != 0:
                i += 1
                temp = applicants[randrange(applicant_num)]
                if i > applicant_num * len(trainee_list) * eyes * 10000:
                   return(False) 
            rev_list.append(temp)
            applicant_counts[temp] += 1
            reviewer_counts[trainee] += 1
            applicant_dict[temp].append(trainee)
            reviewer_dict[trainee].append(temp)

    # faculty
    i = 0
    for faculty in faculty_list:
        rev_list = []
        if faculty in min_list:
            assign_num = min_num
        elif faculty in max_list:
            assign_num = min_num + 1
        for x in range(assign_num):
            i += 1
            temp = applicants[randrange(applicant_num)]
            while temp in rev_list or applicant_counts[temp] == eyes:
                i += 1
                temp = applicants[randrange(applicant_num)]
                if i > applicant_num * len(reviewer_list) * eyes * 10000:
                   return(False)
            rev_list.append(temp)
            applicant_counts[temp] += 1
            reviewer_counts[faculty] += 1
            applicant_dict[temp].append(faculty)
            reviewer_dict[faculty].append(temp)
    return(applicant_dict, reviewer_dict)

attempt = False
while attempt == False:
    attempt = assign_reviewer()
applicant_dict, reviewer_dict = attempt

## ------------------------------ ##
## Print/save results             ##
## ------------------------------ ##

# save table of applicant: reviewers
with open("applicants_reviewers.csv","w") as output:
  output.write("Applicant,Reviewers" + ","*(eyes-1) + "\n")
  print("\n\nApplicant\tReviewers")
  for applicant in applicant_dict:
      line = applicant
      for reviewer in applicant_dict[applicant]:
          line += "," + reviewer
      output.write(line + "\n")
      print(line.replace(",","\t"))
files.download("applicants_reviewers.csv")


# save table(s) of reviewer: applicants
with open("reviewer_applicants.csv","w") as output, open("reviewer_applicants.txt","w") as txt:
  output.write("Reviewer,Applicants" + ","*(min_num) + "\n")
  print("\n\nReviewer\tApplicants")
  for reviewer in reviewer_dict:
      line = reviewer
      txt.write(reviewer + "\n")
      for applicant in reviewer_dict[reviewer]:
          line += "," + applicant
          txt.write("\t" + applicant + "\n")
      while line.count(",") < min_num+1:
        line += ","
      output.write(line + "\n")
      print(line.replace(",","\t"))
files.download("reviewer_applicants.csv")
files.download("reviewer_applicants.txt")



Applicant	Reviewers
A B	Yessica Santana	Tianyi  Mao	Larry Trussell
C D	Nina Luong	Henrique vonGersdorf	Eric Schnell
E F	Nina Luong	Larry Trussell	Kelly Monk
G H	Nina Luong	Gary Westbrook	Kelly Monk
I J	Marissa Co	Kevin Wright	Henrique vonGersdorf
K L	Jennifer Jahncke	Kevin Wright	Henrique vonGersdorf
M N	Marissa Co	Larry Trussell	Henrique vonGersdorf
O P	Jennifer Jahncke	Eric Schnell	Kelly Monk
Q R	Marissa Co	Eric Schnell	Kelly Monk
S T	Jennifer Jahncke	Gary Westbrook	Kevin Wright
U V	Yessica Santana	Gary Westbrook	Tianyi  Mao
W X	Jennifer Jahncke	Tianyi  Mao	Eric Schnell
Y Z	Yessica Santana	Tianyi  Mao	Kevin Wright


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



Reviewer	Applicants
Gary Westbrook	G H	S T	U V	
Tianyi  Mao	A B	U V	W X	Y Z
Kevin Wright	I J	K L	Y Z	S T
Larry Trussell	A B	E F	M N	
Henrique vonGersdorf	I J	C D	M N	K L
Eric Schnell	C D	O P	W X	Q R
Kelly Monk	Q R	O P	E F	G H
Yessica Santana	A B	Y Z	U V	
Nina Luong	C D	E F	G H	
Jennifer Jahncke	O P	W X	S T	K L
Marissa Co	I J	M N	Q R	


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>